# Anosmia project analysis notebook

This notebook contains the analysis of the data collected by the Anosmia project.


***By: Sid Rafilson, Casey Lennon-Jones***

***PI: Matt Smear***




### Imports and helper functions

In [1]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from scipy import signal
import os


def load_sniff(sniff_file: str, num_samples: int = -1, start = 0, stop = 0, nchannels: int = 8, ch: int = 8) -> np.array:
    '''
    Load a specific channel from a binary sniff data file into a NumPy array.

    This function reads a binary file containing multi-channel sniff data, processes it, 
    and extracts data from a specified channel.

    Parameters:
    sniff_file (str): The path to the binary file containing sniff data.
    num_samples (int): The number of samples to read from the file.
    nchannels (int, optional): The number of channels in the sniff data. Defaults to 8.
    ch (int, optional): The channel number to extract. Defaults to 8. Channels are
                        indexed starting from 1.

    Returns:
    np.array: A NumPy array containing the data from the specified channel.

    Notes:
    - The number of samples specified is dynamic for debugging but can easily fit the entire data set with num_samples = -1.
    - Channel numbers start from 1. For instance, ch = 1 will extract the first channel.
    '''
    
    # reading in binary data
    num_samples = num_samples * nchannels
    sniff_bin = np.fromfile(sniff_file, dtype=np.uint16, count=num_samples)

    # ensuring equal samples from each channel
    num_complete_sets = len(sniff_bin) // nchannels
    sniff_bin = sniff_bin[:num_complete_sets * nchannels]

    # reshaping data and extracting channel which corresponds to sniff voltage
    sniff_bin = np.reshape(sniff_bin, (nchannels, -1), order='F')
    ch_index = ch - 1
    sniff = sniff_bin[ch_index, :]

    # removing start seconds from beggining, and stop from end of signal; default is 0
    start = start * 30000
    stop = stop * 30000
    if stop == 0:
        sniff = sniff[start:]  
    else:
        sniff = sniff[start: -stop]

    return sniff
    


def load_ephys(ephys_file: str, num_samples: int = -1, start = 0, stop = 0, nchannels: int = 16) -> np.array:
    '''
    Load and reshape binary electrophysiology data into a NumPy array.

    This function is designed to read binary files containing electrophysiology 
    (ephys) data. It loads the specified number of samples from the file and 
    reshapes them into a 2D NumPy array, where each row represents a channel.

    Parameters:
    ephys_file (str): Path to the binary file containing electrophysiology data.
    num_samples (int): Number of samples to read from the file.
    nchannels (int, optional): Number of channels in the ephys data. Defaults to 16.

    Returns:
    np.array: A 2D NumPy array of the electrophysiology data, reshaped into
              (nchannels, number_of_samples_per_channel).
    '''
  
    # reading in binary data
    num_samples = num_samples * nchannels
    ephys_bin = np.fromfile(ephys_file, dtype=np.uint32, count = num_samples)
    
    # ensuring equal samples from each channel
    num_complete_sets = len(ephys_bin) // nchannels
    ephys_bin = ephys_bin[:num_complete_sets * nchannels]

    # reshape 1d array into nchannels x num_samples NumPy array
    ephys_data = np.reshape(ephys_bin, (nchannels, -1), order='F')

    # removing start seconds from beggining, and stop from end of signal; default is 0
    start = start * 30000
    stop = stop * 30000

    if stop == 0:
        ephys = ephys_data[:, start:]
    else:
        ephys = ephys_data[:, start: -stop]

    return ephys



def resample_sniff(sniff: np.array, original_rate = 30000, target_rate = 1000) -> np.array:
    '''
    Resample a sniff signal from an original rate to a target rate.

    This function applies a decimation process to a sniff signal, which is useful in 
    situations where lower sampling rates are sufficient or desired for analysis. The 
    decimation is performed using a Finite Impulse Response (FIR) filter with a Hamming 
    window to reduce aliasing effects.

    Parameters:
    sniff (np.array): The sniff signal to be resampled, represented as a NumPy array.
    original_rate (int, optional): The original sampling rate of the signal in Hz. Defaults to 30000 Hz.
    target_rate (int, optional): The desired sampling rate of the signal in Hz. Defaults to 1000 Hz.

    Returns:
    np.array: The resampled sniff signal.
    '''

    # calculating resample factor
    resample_factor = original_rate // target_rate

    # calculating new length of resampled signal
    if 0 != sniff.shape[0] % resample_factor:
        sniff = sniff[:-(sniff.shape[0] % resample_factor)]

    # calculating new length of resampled signal
    print(f'Resampling from length {sniff.shape[0]}')
    new_length = sniff.shape[0]//resample_factor

    # applying decimation to the signal
    resampled_sniff = np.zeros(new_length)
    resampled_sniff = signal.decimate(sniff, resample_factor, ftype = 'fir')

    return resampled_sniff
    


def resample_ephys(ephys: np.array, nchannels = 16, original_rate = 30000, target_rate = 1000) -> np.array:
    '''
    Resample multi-channel electrophysiology (ephys) data from an original sampling rate to a target rate.

    This function applies a decimation process to each channel of a multi-channel ephys signal. 
    It uses a 30-point Finite Impulse Response (FIR) filter with a Hamming window to mitigate 
    aliasing effects during resampling.

    Parameters:
    ephys (np.array): A 2D NumPy array representing the ephys data, with shape (nchannels, number_of_samples).
    nchannels (int, optional): Number of channels in the ephys data. Defaults to 16.
    original_rate (int, optional): Original sampling rate of the ephys data in Hz. Defaults to 30000 Hz.
    target_rate (int, optional): Target sampling rate in Hz. Defaults to 1000 Hz.

    Returns:
    np.array: A 2D NumPy array of the resampled ephys data, with the same number of channels 
              and a reduced number of samples per channel.
    '''

    # calculating resample factor
    resample_factor = original_rate // target_rate

    # removing samples from the end of the signal if the length is not divisible by the resample factor
    if 0 != ephys.shape[1] % resample_factor:
        ephys = ephys[:, :-(ephys.shape[1] % resample_factor)]

    # calculating new length of resampled signal
    print(f'Resampling Ephys from length {ephys.shape[1]}')
    new_length = ephys.shape[1]//resample_factor

    # applying decimation to the signal
    resampled_ephys = np.zeros((nchannels, new_length))
    for ch in range(nchannels):
        resampled_ephys[ch, :] = signal.decimate(ephys[ch,:], resample_factor, ftype = 'fir')

    return resampled_ephys



def find_inhales(data: np.array, window_length = 51, polyorder = 5, min_peak_prominance = 75) -> np.array:
    '''
    Smooth a  signal using the Savitzky-Golay method and locate inhalation times using peak finding.

    This function first applies a Savitzky-Golay filter to smooth the input signal. 
    It then uses a peak finding algorithm to identify the times of inhalations, which are 
    indicated by prominent peaks in the smoothed signal. Optionally, the function can also 
    plot the original and smoothed signals along with the identified peaks.

    Parameters:
    data (np.array): The signal to be processed, represented as a NumPy array.
    window_length (int, optional): The length of the filter window. Defaults to 101.
    polyorder (int, optional): The order of the polynomial used to fit the samples. Defaults to 9.
    min_peak_prominance (int, optional): The minimum prominence of a peak to be considered 
                                         an inhalation. Defaults to 50.
    show (bool, optional): If True, display a plot of the original and smoothed signals with peaks. 
                           Defaults to False.

    Returns:
    tuple: A tuple containing two elements:
           - locs (np.array): An array of indices where inhalation peaks are located.
           - smoothed_sniff (np.array): The smoothed sniff signal.
    '''

    smoothed_data = signal.savgol_filter(data, window_length, polyorder)
    locs, properties = signal.find_peaks(smoothed_data, height = (None, None), prominence = min_peak_prominance)
    

    return locs, smoothed_data, properties



def sniff_lock_lfp(locs: np.array, ephys: np.array, window_size = 1000, nsniffs = 512, beg = 3000, method = 'zscore') -> np.array:
    '''
    Aligns local field potential (LFP) signals with sniff inhalation times and constructs a 3D array of z-scored LFP activity.

    This function identifies segments of LFP signals corresponding to inhalation times (specified by 'locs') and 
    standardizes these segments across channels. The output is a 3D array where each 'slice' corresponds to the LFP 
    activity surrounding a single sniff event, with data from all channels.

    Parameters:
    locs (np.array): Array of sniff inhalation times (indices).
    ephys (np.array): 2D array of electrophysiological data with shape (nchannels, number_of_samples).
    nchannels (int, optional): Number of channels in the ephys data. Defaults to 16.
    window_size (int, optional): The size of the window around each sniff event to consider for LFP activity. Defaults to 1000.
    nsniffs (int, optional): Number of sniff events to process. Defaults to 512.
    beg (int, optional): Starting index to begin looking for sniff events. Defaults to 3000.

    Returns:
    sniff_activity (np.array): A 3D NumPy array with shape (nsniffs, window_size, nchannels). Each 'slice' of this array 
              represents the z-scored LFP activity around a single sniff event for all channels.
    loc_set (np.array): An array of indices where inhalation peaks are located.

    Raises:
    ValueError: If the 'locs' array does not contain enough data after the specified 'beg' index for the required number of sniffs.
    '''


    # finding number of channels
    nchannels = ephys.shape[0]


    # finding the set of inhalation times to use
    if nsniffs == 'all':
        loc_set = locs[5:-5]
        nsniffs = len(loc_set)
    elif isinstance(nsniffs, int):
        first_loc = np.argmax(locs >= beg)
        loc_set = locs[first_loc: first_loc + nsniffs]
    else:
        raise ValueError("nsniffs must be either 'all' or an integer.")

    # checking if locs array has enough data for the specified range
    if isinstance(nsniffs, int):
        if len(loc_set) < nsniffs:
            raise ValueError("locs array does not have enough data for the specified range.")
        
    # propogates an nx2 array containing times half the window size in both directions from inhalation times
    windows = np.zeros((nsniffs, 2), dtype=int)
    for ii in range(nsniffs):
        win_beg = loc_set[ii] - round(window_size/2)
        win_end = loc_set[ii] + round(window_size/2)
        windows[ii] = [win_beg, win_end]

    if method == 'zscore':
        # finds and saves zscored ephys data from each channel for each inhalaion locked time window
        sniff_activity = np.zeros((nchannels, nsniffs, window_size))
        for ii in range(nsniffs):
            for ch in range(nchannels):
                win_beg, win_end = windows[ii]
                data = ephys[ch, win_beg:win_end]
                data_mean = np.mean(data)
                data_std = np.std(data)
                zscore_data = (data - data_mean) / data_std
                if len(data) < window_size:
                    data = np.pad(data, (0, window_size - len(data)), mode = 'constant', constant_values = 0)
                    print('!!! padding !!!')
                sniff_activity[ch,ii,:] = zscore_data

    elif method == 'none':
        sniff_activity = np.zeros((nchannels, nsniffs, window_size))
        for ii in range(nsniffs):
            for ch in range(nchannels):
                win_beg, win_end = windows[ii]
                data = ephys[ch, win_beg:win_end]
                if len(data) < window_size:
                    data = np.pad(data, (0, window_size - len(data)), mode = 'constant', constant_values = 0)
                    print('!!! padding !!!')
                sniff_activity[ch,ii,:] = data

    return sniff_activity, loc_set



def sort_lfp(sniff_activity, locs):
    '''sorts the sniff locked lfp trace by sniff frequency'''

    # finding data shape
    nchannels = sniff_activity.shape[0]
    nsniffs = sniff_activity.shape[1]
    window_size = sniff_activity.shape[2]
    
    sorted_activity = np.zeros((nchannels, nsniffs-1, window_size))
    
    # finding sniff frequencies by inhalation time differences (we lose the last sniff)
    freqs = np.diff(locs)

    # sorting the ephys data and frequency values according to these times
    sort_indices = np.argsort(freqs)
    sorted_activity[:, :, :] = sniff_activity[:, sort_indices, :]
    sorted_freqs = freqs[sort_indices]
    sorted_freqs = 1 / (sorted_freqs / 1000)

    return sorted_activity, sorted_freqs



### Loading the data

In [2]:
sniff_file = r"E:\anosmia\data\PRE1\20240711_5001E_PRE1_ADC.bin"
ephys_file = r"E:\anosmia\data\PRE1\20240711_5001E_PRE1_Ephys.bin"

# loading data
sniff = load_sniff(sniff_file)
spikes = load_ephys(ephys_file)
LFP = resample_ephys(spikes)
sniff_resampled = resample_sniff(sniff)

# filtering spikes signal
sos = signal.butter(10, 1_000, 'high', fs = 30_000, output = 'sos')
spikes = signal.sosfiltfilt(sos, spikes, axis = 1)

# time vectors
time = np.arange(0, sniff_resampled.shape[0], 1) / 1000
time_spikes = time / 30


print(f'length of sniff signal: {sniff.shape[0]}\nlength of resampled sniff signal: {sniff_resampled.shape[0]}\nlength of spikes signal: {spikes.shape[1]}\nlength of resampled spikes signal: {LFP.shape[1]}')



Resampling Ephys from length 110270400
Resampling from length 110270400
length of sniff signal: 110270400
length of resampled sniff signal: 3675680
length of spikes signal: 110270400
length of resampled spikes signal: 3675680


### Filtering the data

In [3]:
filters = {'theta': (0.5, 24), 'beta': (24, 40), 'gamma': (65, 85)}

for key, value in tqdm(filters.items(), desc='Filtering LFP signals'):
    sos = signal.butter(10, value, 'band', fs=1000, output='sos')
    filtered = signal.sosfiltfilt(sos, LFP, axis=1)
    globals()[key] = filtered
    del sos

gamma_envelope = np.abs(signal.hilbert(gamma, axis=1))



Filtering LFP signals: 100%|██████████| 3/3 [00:06<00:00,  2.02s/it]


### Plotting the LFPs with sniff

In [8]:
save_path = r"E:\anosmia\figs"

First plotting the raw LFPs

In [11]:
start = 10_000
end = 15_000

fig_path = os.path.join(save_path, 'sniff_lfp')
if not os.path.exists(fig_path):
    os.makedirs(fig_path)


for j in range(10):

    start = start + 100_000
    end = end + 100_000

    fig, axs = plt.subplots(17, 1, figsize=(17, 30), sharex=True)
    sns.despine()
    axs[0].plot(time[start:end], sniff_resampled[start:end], color = 'dodgerblue')
    axs[0].set_title('Sniff Data')
    axs[0].set_ylabel('Voltage (mV)')
    for i in range(1, 17):
        axs[i].plot(time[start:end], LFP[i-1, start:end], color = 'purple')
        axs[i].set_title(f'Channel {i}')
        axs[i].set_ylabel('Voltage (mV)')
    axs[16].set_xlabel('Time (s)')
    plt.tight_layout()
    plt.savefig(f'{fig_path}\\sniff_lfp{j}.png', dpi=300)
    plt.close()


Plotting the low frequency Theta (0.5 - 24Hz) LFPs

In [13]:
fig_path = os.path.join(save_path, 'sniff_theta')
if not os.path.exists(fig_path):
    os.makedirs(fig_path)

for j in range(10):

    start = start + 100_000
    end = end + 100_000

    fig, axs = plt.subplots(17, 1, figsize=(17, 30), sharex=True)
    sns.despine()
    axs[0].plot(time[start:end], sniff_resampled[start:end], color = 'dodgerblue')
    axs[0].set_title('Sniff Data')
    axs[0].set_ylabel('Voltage (mV)')
    for i in range(1, 17):
        axs[i].plot(time[start:end], theta[i-1, start:end], color = 'purple')
        axs[i].set_title(f'Channel {i}')
        axs[i].set_ylabel('Voltage (mV)')
    axs[16].set_xlabel('Time (s)')
    plt.tight_layout()
    plt.savefig(f'{fig_path}\\sniff_theta{j}.png', dpi=300)
    plt.close()

Plotting the beta (24 - 40Hz) LFPs

In [14]:
fig_path = os.path.join(save_path, 'sniff_beta')
if not os.path.exists(fig_path):
    os.makedirs(fig_path)



for j in range(10):

    start = start + 100_000
    end = end + 100_000

    fig, axs = plt.subplots(17, 1, figsize=(17, 30), sharex=True)
    sns.despine()
    axs[0].plot(time[start:end], sniff_resampled[start:end], color = 'dodgerblue')
    axs[0].set_title('Sniff Data')
    axs[0].set_ylabel('Voltage (mV)')
    for i in range(1, 17):
        axs[i].plot(time[start:end], beta[i-1, start:end], color = 'purple')
        axs[i].set_title(f'Channel {i}')
        axs[i].set_ylabel('Voltage (mV)')
    axs[16].set_xlabel('Time (s)')
    plt.tight_layout()
    plt.savefig(f'{fig_path}\\sniff_beta{j}.png', dpi=300)
    plt.close()


Plotting the gamma band (65 - 85Hz) LFPs and its amplitude envelope with the sniff

In [20]:
fig_path = os.path.join(save_path, 'sniff_gamma')
if not os.path.exists(fig_path):
    os.makedirs(fig_path)

start = 10_000
end = 15_000

for j in range(10):

    start = start + 100_000
    end = end + 100_000

    fig, axs = plt.subplots(17, 1, figsize=(17, 30), sharex=True)
    sns.despine()
    axs[0].plot(time[start:end], sniff_resampled[start:end], color = 'dodgerblue')
    axs[0].set_title('Sniff Data')
    axs[0].set_ylabel('Voltage (mV)')
    for i in range(1, 17):
        axs[i].plot(time[start:end], gamma[i-1, start:end], color = 'purple')
        axs[i].plot(time[start:end], gamma_envelope[i-1, start:end], color = 'red')
        axs[i].set_title(f'Channel {i}')
        axs[i].set_ylabel('Voltage (mV)')
    axs[16].set_xlabel('Time (s)')
    plt.tight_layout()
    plt.savefig(f'{fig_path}\\sniff_gamma{j}.png', dpi=300)
    plt.close()


### Plotting the Spikes with sniff

In [19]:
fig_path = os.path.join(save_path, 'sniff_spikes')
if not os.path.exists(fig_path):
    os.makedirs(fig_path)

start = 100_000
end = 190_000

for j in range(10):

    start = start + 100_000
    end = end + 100_000

    fig, axs = plt.subplots(17, 1, figsize=(17, 30), sharex=True)
    sns.despine()
    axs[0].plot(time_spikes[start:end], sniff[start:end], color = 'dodgerblue')
    axs[0].set_title('Sniff Data')
    axs[0].set_ylabel('Voltage (mV)')
    for i in range(1, 17):
        axs[i].plot(time_spikes[start:end], spikes[i-1, start:end], color = 'purple')
        axs[i].set_title(f'Channel {i}')
        axs[i].set_ylabel('Voltage (mV)')
    axs[16].set_xlabel('Time (s)')
    plt.tight_layout()
    plt.savefig(f'{fig_path}\\sniff_spikes{j}.png', dpi=300)
    plt.close()


### Finding inhalation times and plotting the sniff frequency histogram

In [6]:
locs, _, _ = find_inhales(sniff_resampled)
freqs = 1000 / np.diff(locs)
freqs = freqs[(freqs < 16) & (freqs > 0.5)]

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(freqs, bins = 50, kde = True)
plt.title('Sniff Frequency Distribution')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Count')
plt.savefig(f'{save_path}\\sniff_freqs.png', dpi=300)
plt.close()

### Building sniff aligned LFP rasters

In [31]:
activity, locs = sniff_lock_lfp(locs, LFP)
sorted_activity, sorted_freqs = sort_lfp(activity, locs)

raster_path = os.path.join(save_path, 'raster_plots_raw')
if not os.path.exists(raster_path):
    os.makedirs(raster_path)

vmax = np.percentile(np.abs(sorted_activity), 95)
for i in range(16):
    plt.figure(figsize=(10, 6))
    sns.heatmap(sorted_activity[i, :, :], cmap='seismic', robust=True, vmax=vmax, vmin=-vmax)
    plt.title(f'Channel {i+1}')
    plt.xticks([])
    plt.yticks([])
    plt.savefig(f'{raster_path}\\channel_{i+1}.png', dpi=300)
    plt.close()







No such comm: 6ad0b8d5886249c4b5bd3438e45c639e


In [32]:
activity, locs = sniff_lock_lfp(locs, theta)
sorted_activity, sorted_freqs = sort_lfp(activity, locs)

raster_path = os.path.join(save_path, 'raster_plots_theta')
if not os.path.exists(raster_path):
    os.makedirs(raster_path)

vmax = np.percentile(np.abs(sorted_activity), 95)
for i in range(16):
    plt.figure(figsize=(10, 6))
    sns.heatmap(sorted_activity[i, :, :], cmap='seismic', robust=True, vmax=vmax, vmin=-vmax)
    plt.title(f'Channel {i+1}')
    plt.xticks([])
    plt.yticks([])
    plt.savefig(f'{raster_path}\\channel_{i+1}.png', dpi=300)
    plt.close()


No such comm: a692bcf679444c5d9a3da78fef247c47


In [33]:
activity, locs = sniff_lock_lfp(locs, beta)
sorted_activity, sorted_freqs = sort_lfp(activity, locs)

raster_path = os.path.join(save_path, 'raster_plots_beta')
if not os.path.exists(raster_path):
    os.makedirs(raster_path)

vmax = np.percentile(np.abs(sorted_activity), 95)

for i in range(16):
    plt.figure(figsize=(10, 6))
    sns.heatmap(sorted_activity[i, :, :], cmap='seismic', robust=True, vmax=vmax, vmin=-vmax)
    plt.title(f'Channel {i+1}')
    plt.xticks([])
    plt.yticks([])
    plt.savefig(f'{raster_path}\\channel_{i+1}.png', dpi=300)
    plt.close()


No such comm: 5658175ea51245b9a8dc1acddb0c12b1


In [34]:
activity, locs = sniff_lock_lfp(locs, gamma)
sorted_activity, sorted_freqs = sort_lfp(activity, locs)

raster_path = os.path.join(save_path, 'raster_plots_gamma')
if not os.path.exists(raster_path):
    os.makedirs(raster_path)

vmax = np.percentile(np.abs(sorted_activity), 95)

for i in range(16):
    plt.figure(figsize=(10, 6))
    sns.heatmap(sorted_activity[i, :, :], cmap='seismic', robust=True, vmax=vmax, vmin=-vmax)
    plt.title(f'Channel {i+1}')
    plt.xticks([])
    plt.yticks([])
    plt.savefig(f'{raster_path}\\channel_{i+1}.png', dpi=300)
    plt.close()


No such comm: 875a213db9c74b28a06a7d4174a3121d


In [35]:
activity, locs = sniff_lock_lfp(locs, gamma_envelope)
sorted_activity, sorted_freqs = sort_lfp(activity, locs)

raster_path = os.path.join(save_path, 'raster_plots_gamma_envelope')
if not os.path.exists(raster_path):
    os.makedirs(raster_path)

vmax = np.percentile(np.abs(sorted_activity), 95)


for i in range(16):
    plt.figure(figsize=(10, 6))
    sns.heatmap(sorted_activity[i, :, :], cmap='seismic', robust=True, vmax=vmax, vmin=-vmax)
    plt.title(f'Channel {i+1}')
    plt.xticks([])
    plt.yticks([])
    plt.savefig(f'{raster_path}\\channel_{i+1}.png', dpi=300)
    plt.close()



No such comm: f187aabf63a84d4cb2b7369095d20a81
